<a href="https://colab.research.google.com/github/lauramauricio/election-prediction-webapp/blob/main/Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [50]:
!pip install dash pandas scikit-learn plotly

from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px
import requests
import io


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report


In [2]:
import numpy as np

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

In [22]:
data_path = "https://raw.githubusercontent.com/lauramauricio/election-prediction-webapp/efe5785ebf31c3bf48d528e9aa2b6ebc7fa46d29/merged_dataset.csv"
df = pd.read_csv(data_path)

<ipython-input-22-2db63ce4b8c9>:2: DtypeWarning: Columns (11,14,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [55]:
df["vdn1b"].unique()

array([nan, 'fdp/prd', 'pda/pdt', 'sps/pss', 'other parties',
       'rep. (& vigil.)', 'cvp/pdc', 'svp/udc', 'lps/pls', 'evp/pep',
       'ldu/adi', 'csp/pcs', 'sd/ds', 'poch', 'voted blank', 'gps/pes',
       'fps/psl', 'fga/avf', 'right parties', 'left parties',
       'other comments', 'edu/udf', 'sol.', 'lega', 'psa (psu)',
       'centre parties', 'bdp', "GLP/Vert'libéraux", 'mcg'], dtype=object)

In [23]:
column_drop = ["med1"]
df_clean = df.drop(column_drop, axis=1)

In [ ]:
df_clean =

In [45]:
# Group by year and calculate the share of voter for each party
yearly_party_share = df.groupby('year')['vdn1b'].value_counts(normalize=True).rename('share').reset_index()

print(yearly_party_share)  # Display the first few rows of the result

     year          vdn1b     share
0    1971        sps/pss  0.207490
1    1971        cvp/pdc  0.168016
2    1971        fdp/prd  0.159919
3    1971        svp/udc  0.134615
4    1971        ldu/adi  0.119433
..    ...            ...       ...
194  2019           lega  0.011376
195  2019  other parties  0.007438
196  2019    voted blank  0.002844
197  2019        edu/udf  0.002844
198  2019        csp/pcs  0.001094

[199 rows x 3 columns]


In [49]:
# Filter for the "sps/pss" party
sps_pss_share = yearly_party_share[yearly_party_share['vdn1b'] == 'sps/pss'][['year', 'share']]

# Calculate 200 divided by 'total_gewahlt' for each year
df['total_gewahlt_proportion'] = df['total_gewahlt']/200

# Calculate the average proportion per year
total_gewahlt_proportion_by_year = df.groupby('year')['total_gewahlt_proportion'].mean().reset_index()

# Merge the party share with the total_gewahlt proportion by year
comparison_df = pd.merge(sps_pss_share, total_gewahlt_proportion_by_year, on='year')
comparison_df.rename(columns={'share': 'sps_pss_share', 'total_gewahlt_proportion': 'total_gewahlt_proportion'}, inplace=True)

print("Comparison DataFrame:")
print(comparison_df)

Comparison DataFrame:
    year  sps_pss_share  total_gewahlt_proportion
0   1971       0.207490                     0.230
1   1975       0.284330                     0.275
2   1979       0.241739                     0.255
3   1987       0.155779                     0.205
4   1991       0.169463                     0.205
5   1995       0.320154                     0.270
6   1999       0.292002                     0.255
7   2003       0.290357                     0.260
8   2007       0.214362                     0.215
9   2011       0.238079                     0.230
10  2015       0.207882                     0.215
11  2019       0.169328                     0.195


In [24]:
# Replace "NaN" values with 0 for both numeric and categorical data
df_clean = df_clean.fillna(0)

In [25]:
df_clean.head()


,year,userid,sex,age,educ,income,religion,sg1,sg9,sc1,...,vp1,vdn1b,pid1,pid2b,trust1,weighttot,party,total_gewahlt,total_men,total_women
0,1971,1,male,33.0,primary school,0,cath,French spoken,0,in training/formation,...,yes,0,yes,sps/pss,0,0.936000,SP,46.0,43.0,3.0
1,1971,2,male,56.0,compulsory education,0,other,French spoken,0,full-time,...,yes,0,yes,sps/pss,6.666666507720947,0.936000,SP,46.0,43.0,3.0
2,1971,3,male,49.0,primary school,0,cath,French spoken,0,full-time,...,0,0,no,no party identification,6.666666507720947,1.000000,SP,46.0,43.0,3.0
3,1971,4,female,60.0,primary school,0,prot,French spoken,0,in household,...,no,0,no,no party identification,0,1.099000,SP,46.0,43.0,3.0
4,1971,5,female,39.0,compulsory education,0,prot,French spoken,0,without profession,...,yes,fdp/prd,no,no party identification,3.3333332538604736,1.277453,SP,46.0,43.0,3.0


In [26]:
# Identify categorical columns
categorical_cols = df_clean.select_dtypes(include=['object']).columns

# Encode categorical columns using LabelEncoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))
    label_encoders[col] = le

In [32]:
df_clean = df_clean.drop('year', axis=1)
df_clean

,userid,sex,age,educ,income,religion,sg1,sg9,sc1,sc7a,...,vp1,vdn1b,pid1,pid2b,trust1,weighttot,party,total_gewahlt,total_men,total_women
0,1,2,33.0,6,2,1,1,0,5,0,...,2,0,2,27,0,0.936000,0,46.0,43.0,3.0
1,2,2,56.0,2,2,3,1,0,2,0,...,2,0,2,27,9,0.936000,0,46.0,43.0,3.0
2,3,2,49.0,6,2,1,1,0,2,0,...,0,0,1,17,9,1.000000,0,46.0,43.0,3.0
3,4,1,60.0,6,2,4,1,0,4,0,...,1,0,1,17,0,1.099000,0,46.0,43.0,3.0
4,5,1,39.0,2,2,4,1,0,10,0,...,2,8,1,17,5,1.277453,0,46.0,43.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43664,43665,2,83.0,9,4,1,2,1,8,0,...,1,0,2,5,0,4.796863,0,39.0,14.0,25.0
43665,43666,1,42.0,7,0,4,2,2,7,0,...,2,1,2,1,0,0.607058,0,39.0,14.0,25.0
43666,43667,1,46.0,7,5,1,1,2,9,0,...,2,26,2,27,0,0.084512,0,39.0,14.0,25.0
43667,43668,2,34.0,9,0,4,2,2,2,0,...,1,0,1,17,0,0.804412,0,39.0,14.0,25.0


In [28]:
df.head()

,year,userid,sex,age,educ,income,religion,sg1,sg9,sc1,...,vdn1b,pid1,pid2b,trust1,med1,weighttot,party,total_gewahlt,total_men,total_women
0,1971,1,male,33.0,primary school,NaN,cath,French spoken,NaN,in training/formation,...,NaN,yes,sps/pss,NaN,NaN,0.936000,SP,46.0,43.0,3.0
1,1971,2,male,56.0,compulsory education,NaN,other,French spoken,NaN,full-time,...,NaN,yes,sps/pss,6.666666507720947,NaN,0.936000,SP,46.0,43.0,3.0
2,1971,3,male,49.0,primary school,NaN,cath,French spoken,NaN,full-time,...,NaN,no,no party identification,6.666666507720947,NaN,1.000000,SP,46.0,43.0,3.0
3,1971,4,female,60.0,primary school,NaN,prot,French spoken,NaN,in household,...,NaN,no,no party identification,NaN,NaN,1.099000,SP,46.0,43.0,3.0
4,1971,5,female,39.0,compulsory education,NaN,prot,French spoken,NaN,without profession,...,fdp/prd,no,no party identification,3.3333332538604736,NaN,1.277453,SP,46.0,43.0,3.0


In [54]:
df_clean["vdn1b"].unique()

array([ 0,  8, 19, 26, 18, 22,  5, 27, 15,  7, 12,  4, 24, 20, 28, 11, 10,
        9, 23, 13, 17,  6, 25, 14, 21,  3,  2,  1, 16])

In [33]:
# Split the data into features and target
X = df_clean.drop('total_gewahlt', axis=1)
y = df_clean['total_gewahlt']

# Train the Random Forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
predictions = rf_model.predict(X_test)


In [51]:
# Split the data into features and target
X = df_clean.drop('vdn1b', axis=1)
y = df_clean['vdn1b']

# Train the Random Forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
predictions = rf_model.predict(X_test)

In [53]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      4921
           1       0.57      0.41      0.48       187
           2       0.50      0.12      0.20        99
           3       0.00      0.00      0.00         2
           4       0.79      0.39      0.52        49
           5       0.69      0.72      0.71      1057
           6       0.79      0.25      0.38        44
           7       0.61      0.23      0.34       116
           8       0.64      0.75      0.69      1508
           9       1.00      0.06      0.12        32
          10       0.43      0.10      0.16        31
          11       0.68      0.65      0.67       643
          12       0.89      0.69      0.78        90
          13       0.50      0.04      0.08        24
          14       0.81      0.67      0.73        92
          15       0.80      0.53      0.63       139
          16       0.38      0.30      0.33        27
          17       0.41    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# Print regression metrics
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print('R2 Score:', r2_score(y_test, predictions))

Mean Squared Error: 0.0008778795511792994
R2 Score: 0.9999711597630168


In [36]:
from sklearn.model_selection import cross_val_score

In [37]:
# Evaluate the model using cross-validation
mse_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)

print("Cross-Validated RMSE Scores:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())
print("Standard Deviation of RMSE:", rmse_scores.std())

# R² scores
r2_scores = cross_val_score(rf_model, X, y, cv=5, scoring='r2')
print("\nCross-Validated R² Scores:", r2_scores)
print("Mean R²:", r2_scores.mean())
print("Standard Deviation of R²:", r2_scores.std())

Cross-Validated RMSE Scores: [7.02148383 1.01615557 0.74535012 2.12714149 3.49418388]
Mean RMSE: 2.880862977831854
Standard Deviation of RMSE: 2.2858936354314254

Cross-Validated R² Scores: [-0.66573713  0.50051311  0.97088226 -1.01105227 -3.22087965]
Mean R²: -0.6852547371823693
Standard Deviation of R²: 1.4618313036186263
